# CLARITY: SemEval 2026 — Training Notebook

Works on **both Colab (GPU) and local**.

| Config | Model | VRAM | Expected Task2 F1 |
|--------|-------|------|-------------------|
| `deberta_v3_base.yaml` | DeBERTa-v3-base | ~6GB | ~0.48–0.56 |
| `deberta_v3_large.yaml` | DeBERTa-v3-large | ~16GB | ~0.56–0.65 |

## 1. Setup Environment

In [ ]:
# Check GPU
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    props = torch.cuda.get_device_properties(0)
    vram = getattr(props, 'total_memory', getattr(props, 'total_mem', 0))
    print(f"VRAM: {vram / 1e9:.1f} GB")
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    print("MPS (Apple Silicon) available")
else:
    print("CPU only")

In [ ]:
import os, sys

# Detect environment
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

print(f"Environment: {'Colab' if IN_COLAB else 'Local'}")

if IN_COLAB:
    # ═══════════════════════════════════════════════════════════════
    # EDIT THIS: your GitHub repo URL (must push repo first!)
    REPO_URL = "https://github.com/wilsebbis/semeval.git"
    # ═══════════════════════════════════════════════════════════════
    import subprocess
    if not os.path.exists('/content/semeval'):
        subprocess.run(['git', 'clone', REPO_URL, '/content/semeval'], check=True)
    else:
        subprocess.run(['git', '-C', '/content/semeval', 'pull'], check=True)
    os.chdir('/content/semeval')
    !pip install -q -e ".[dev]" 2>&1 | tail -3
else:
    # Running locally — cd to project root
    if os.path.basename(os.getcwd()) == 'notebooks':
        os.chdir(os.path.join(os.getcwd(), '..'))

print(f"Working dir: {os.getcwd()}")

# Verify
from clarity.labels import EVASION_LABELS, CLARITY_LABELS, validate_labels
validate_labels()
print(f"✓ {len(EVASION_LABELS)} evasion labels, {len(CLARITY_LABELS)} clarity labels")

## 2. Prepare Data

In [ ]:
from pathlib import Path

if not Path('data/train.csv').exists():
    !git clone https://huggingface.co/datasets/ailsntua/QEvasion 2>/dev/null || echo "Already cloned"
    !bash scripts/prepare_data.sh
else:
    print("Data already prepared.")

In [ ]:
import pandas as pd

train = pd.read_csv("data/train.csv")
dev = pd.read_csv("data/dev.csv")
print(f"Train: {len(train)} rows | Dev: {len(dev)} rows")
print(f"\nEvasion distribution:")
print(train["evasion_label"].value_counts().to_string())

## 3. Train

In [ ]:
# ── Choose config ────────────────────────────────────────
CONFIG = "configs/deberta_v3_base.yaml"           # T4 OK
# CONFIG = "configs/deberta_v3_large.yaml"        # A100/L4
# CONFIG = "configs/deberta_v3_base_no_weights.yaml"  # ablation
# ─────────────────────────────────────────────────────────

import yaml
with open(CONFIG) as f:
    cfg = yaml.safe_load(f)
print(f"Config: {CONFIG}")
for k, v in sorted(cfg.items()):
    print(f"  {k}: {v}")

In [ ]:
!python -m clarity.train \
    --config {CONFIG} \
    --data data/train.csv \
    --dev data/dev.csv \
    --task evasion

## 4. Evaluate

In [ ]:
import json
from pathlib import Path

output_dir = cfg["output_dir"]
metrics_path = Path(output_dir) / "metrics.json"

if metrics_path.exists():
    with open(metrics_path) as f:
        metrics = json.load(f)
    for m in metrics:
        ep = m.get('epoch', '?')
        ev = m.get('evasion_macro_f1', 0)
        cl = m.get('clarity_macro_f1', 0)
        print(f"  Epoch {ep}: Task2 F1={ev:.4f}, Task1 F1={cl:.4f}")
else:
    print("No metrics found — check training output above.")

## 5. Predict

In [ ]:
CKPT = f"{output_dir}/best_model.pt"
DATA = "data/dev.csv"  # Change to data/test.csv for submission

!python -m clarity.predict \
    --ckpt {CKPT} \
    --data {DATA} \
    --out submissions/predictions.csv \
    --evaluate

In [ ]:
preds = pd.read_csv("submissions/predictions.csv")
print(f"Predictions: {len(preds)} rows")
print(preds["evasion_pred"].value_counts().to_string())

## 6. Ensemble (Optional)

Train 3 seeds, average logits → +1–3 F1 points.

In [ ]:
SEEDS = [42, 123, 2026]

for seed in SEEDS:
    out_dir = f"checkpoints/ensemble_seed{seed}"
    print(f"\n{'='*60}")
    print(f"Training seed {seed} -> {out_dir}")
    print(f"{'='*60}")
    !python -m clarity.train \
        --config {CONFIG} \
        --data data/train.csv \
        --dev data/dev.csv \
        --task evasion \
        --seed {seed} \
        --output_dir {out_dir}

In [ ]:
ckpts = [f"checkpoints/ensemble_seed{s}/best_model.pt" for s in SEEDS]
ckpts_str = " ".join(ckpts)

!python -m clarity.predict \
    --ckpt {ckpts[0]} \
    --ensemble_ckpts {ckpts_str} \
    --data data/dev.csv \
    --out submissions/ensemble_predictions.csv \
    --evaluate

## 7. Download (Colab only)

In [ ]:
if IN_COLAB:
    from google.colab import files
    files.download("submissions/predictions.csv")
else:
    print(f"Results: {os.path.abspath('submissions/predictions.csv')}")